#### Design Pool generation for architecture related features (Step-3)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle

In [2]:
pwd = os.getcwd() # current wroking directory

In [3]:
df = pd.read_csv(pwd+"/Sn_based/Sn_cleaned.csv") # loading Sn based dataset
df.head()

,Ref_DOI_number,Cell_architecture,Cell_flexible,Cell_semitransparent,Substrate_stack_sequence,ETL_stack_sequence,ETL_thickness,ETL_deposition_procedure,Perovskite_dimension_2D,Perovskite_dimension_2D3D_mixture,...,HTL_stack_sequence,HTL_thickness_list,HTL_deposition_procedure,Backcontact_stack_sequence,Backcontact_thickness_list,Backcontact_deposition_procedure,JV_default_Voc,JV_default_Jsc,JV_default_FF,JV_default_PCE
0,10.1002/adfm.201807696,pin,False,False,SLG | ITO,PCBM-60 | BCP,142.0,Spin-coating | Evaporation,True,False,...,PEDOT:PSS,114.0,Spin-coating,Ag,200.0,Evaporation,0.58,21.2,0.633,7.78
1,10.1002/adfm.201807696,pin,False,False,SLG | ITO,PCBM-60 | BCP,142.0,Spin-coating | Evaporation,True,False,...,PEDOT:PSS,114.0,Spin-coating,Ag,200.0,Evaporation,0.60,20.5,0.650,7.95
2,10.1002/adfm.201807696,pin,False,False,SLG | ITO,PCBM-60 | BCP,142.0,Spin-coating | Evaporation,True,False,...,PEDOT:PSS,114.0,Spin-coating,Ag,200.0,Evaporation,0.61,21.0,0.670,8.71
3,10.1002/adfm.201807696,pin,False,False,SLG | ITO,PCBM-60 | BCP,142.0,Spin-coating | Evaporation,True,False,...,PEDOT:PSS,114.0,Spin-coating,Ag,200.0,Evaporation,0.54,20.3,0.630,5.94
4,10.1021/acsenergylett.9b00954,pin,False,False,SLG | ITO,C60 | LiF,31.0,Evaporation | Evaporation,True,False,...,PEDOT:PSS,38.0,Spin-coating,Al,100.0,Evaporation,0.56,18.3,0.493,5.07


In [4]:
df.shape # shape of the Sn-based dataset

(256, 34)

In [6]:
# selected columns
cols = ['Cell_architecture', 'Cell_flexible',
       'Cell_semitransparent', 'Substrate_stack_sequence',
       'ETL_stack_sequence', 'ETL_thickness', 'ETL_deposition_procedure',
       'Perovskite_dimension_2D', 'Perovskite_dimension_2D3D_mixture',
       'Perovskite_dimension_3D',
       'Perovskite_dimension_3D_with_2D_capping_layer',
       'Perovskite_composition_perovskite_ABC3_structure',
       'Perovskite_composition_long_form',
       'Perovskite_thickness',
       'Perovskite_composition_inorganic', 'Perovskite_band_gap',
       'Perovskite_band_gap_graded', 'Perovskite_deposition_procedure',
       'Perovskite_deposition_solvents',
       'Perovskite_deposition_quenching_induced_crystallisation',
       'Perovskite_deposition_thermal_annealing_temperature',
       'Perovskite_deposition_thermal_annealing_time',
       'Perovskite_deposition_solvent_annealing', 'HTL_stack_sequence',
       'HTL_thickness_list', 'HTL_deposition_procedure',
       'Backcontact_stack_sequence', 'Backcontact_thickness_list',
       'Backcontact_deposition_procedure', 'JV_default_PCE']
df = df[cols]
df.shape

(256, 30)

In [7]:
feats = df.iloc[:,:-1] # selecting features
feats.shape # shape of the feature data

(256, 29)

In [9]:
numerical = list(feats.select_dtypes(include="float64")) # numerical features
categorical = list(feats.select_dtypes(include="object")) # categorical features
boolean = list(feats.select_dtypes(include="bool")) # boolean featuers

print("Numerical: ", len(numerical))
print("Categorical: ", len(categorical))
print("Boolean: ", len(boolean))

Numerical:  5
Categorical:  13
Boolean:  11


In [10]:
# architecture related features
feats_arch = [        
            'Cell_architecture', 'Cell_flexible',
            'Cell_semitransparent', 'Substrate_stack_sequence',
            'ETL_stack_sequence',
            'ETL_thickness', 
            'Perovskite_dimension_2D', 'Perovskite_dimension_2D3D_mixture',
            'Perovskite_dimension_3D',
            'Perovskite_dimension_3D_with_2D_capping_layer',
            'Perovskite_composition_perovskite_ABC3_structure',
            'Perovskite_composition_long_form',
            'Perovskite_composition_inorganic',
            'Perovskite_thickness',
            'Perovskite_band_gap', 'Perovskite_band_gap_graded',
            'HTL_stack_sequence',
            'HTL_thickness_list',
            'Backcontact_stack_sequence',
            'Backcontact_thickness_list']
# deposition related features
feats_deposition = [
            'ETL_deposition_procedure',
            'Perovskite_deposition_procedure', 
            'Perovskite_deposition_solvents',
            'Perovskite_deposition_quenching_induced_crystallisation',
            'Perovskite_deposition_thermal_annealing_temperature',
            'Perovskite_deposition_thermal_annealing_time',
            'Perovskite_deposition_solvent_annealing',
            'HTL_deposition_procedure',
            'Backcontact_deposition_procedure'
]
# printing number of features in each group
print(f"Architecture features: {len(feats_arch)}\nDeposition features: {len(feats_deposition)}\nTotal features: {feats.shape[1]}")

Architecture features: 20
Deposition features: 9
Total features: 29


In [11]:
# Number of unique labels on each feature related to device architecture
archi_unique = {}
for f in feats_arch:
    lst = list(df[f].unique())
    archi_unique[f] = lst
for k in archi_unique.keys():
    print(k+" : "+str(len(archi_unique[k])))

Cell_architecture : 2
Cell_flexible : 1
Cell_semitransparent : 1
Substrate_stack_sequence : 2
ETL_stack_sequence : 19
ETL_thickness : 64
Perovskite_dimension_2D : 2
Perovskite_dimension_2D3D_mixture : 2
Perovskite_dimension_3D : 2
Perovskite_dimension_3D_with_2D_capping_layer : 1
Perovskite_composition_perovskite_ABC3_structure : 2
Perovskite_composition_long_form : 80
Perovskite_composition_inorganic : 2
Perovskite_thickness : 55
Perovskite_band_gap : 45
Perovskite_band_gap_graded : 1
HTL_stack_sequence : 20
HTL_thickness_list : 69
Backcontact_stack_sequence : 7
Backcontact_thickness_list : 18


### Design pool for architecture related features

In [12]:
"""feats_arch = [        
                  'Cell_architecture',   
                  'Cell_flexible',       
                  'Cell_semitransparent', 
                  'Substrate_stack_sequence',
                  'ETL_stack_sequence',
                  'ETL_thickness', 
                  'Perovskite_dimension_2D',  
                  'Perovskite_dimension_2D3D_mixture',
                  'Perovskite_dimension_3D', 
                  'Perovskite_dimension_3D_with_2D_capping_layer', 
                  'Perovskite_composition_perovskite_ABC3_structure', 
                  'Perovskite_composition_long_form', 
                  'Perovskite_composition_inorganic', 
                  'Perovskite_thickness', 
                  'Perovskite_band_gap', 
                  'Perovskite_band_gap_graded', 
                  'HTL_stack_sequence', 
                  'HTL_thickness_list', 
                  'Backcontact_stack_sequence', 
                  'Backcontact_thickness_list'
                ]
"""
# Making a list of unique labels in each column
cell_arch_lst = archi_unique['Cell_architecture']
cell_flex_lst = archi_unique['Cell_flexible']
cell_semi_lst = archi_unique['Cell_semitransparent']
subs_lst = archi_unique['Substrate_stack_sequence']
etl_lst = archi_unique['ETL_stack_sequence']
# etl_th_lst = []
per_2d_lst = archi_unique['Perovskite_dimension_2D']
per_2d3d_lst = archi_unique['Perovskite_dimension_2D3D_mixture']
per_3d_lst = archi_unique['Perovskite_dimension_3D']
per_3d2d_cap_lst = archi_unique['Perovskite_dimension_3D_with_2D_capping_layer'] 
per_abc3_lst = archi_unique['Perovskite_composition_perovskite_ABC3_structure']
per_lst = archi_unique['Perovskite_composition_long_form']
per_ino_lst = archi_unique['Perovskite_composition_inorganic']
# per_th_lst = []
# per_band_lst = []
per_band_grad_lst = archi_unique['Perovskite_band_gap_graded']
htl_lst = archi_unique['HTL_stack_sequence']
# htl_th_lst = []
back_lst = archi_unique['Backcontact_stack_sequence']
# back_th = []

cell_arch = []
cell_flex = []
cell_semi = []
subs = []
etl = [] 
# etl_th_lst = [26.0]
per_2d = []
per_2d3d = []
per_3d = []
per_3d2d_cap = []
per_abc3 = []
per = []
per_ino = []
# per_th_lst = [350.0]
# per_band_lst = [1.6]
per_band_grad = []
htl = [] 
# htl_th_lst = [344.0]
back = []
# back_th = [100.0]

# Permutating through each layers of features
for l0 in cell_arch_lst:
    for l1 in cell_flex_lst:
        for l2 in cell_semi_lst:
            for l3 in subs_lst:
                for l4 in etl_lst:
                    for l5 in per_2d_lst:
                        for l6 in per_2d3d_lst:
                            for l7 in per_3d_lst:
                                for l8 in per_3d2d_cap_lst:
                                    for l9 in per_abc3_lst:
                                        for l10 in per_lst:
                                            for l11 in per_ino_lst:
                                                for l12 in per_band_grad_lst:
                                                    for l13 in htl_lst:
                                                        for l14 in back_lst:
                                                            cell_arch.append(l0)
                                                            cell_flex.append(l1)
                                                            cell_semi.append(l2)
                                                            subs.append(l3)
                                                            etl.append(l4)
                                                            per_2d.append(l5)
                                                            per_2d3d.append(l6)
                                                            per_3d.append(l7)
                                                            per_3d2d_cap.append(l8)
                                                            per_abc3.append(l9)
                                                            per.append(l10)
                                                            per_ino.append(l11)
                                                            per_band_grad.append(l12)
                                                            htl.append(l13)
                                                            back.append(l14)

# Create a design pool of all possible combination of arcthitecture related features                                                         
dp_archi = pd.DataFrame(
{   'Cell_architecture': cell_arch,
    'Cell_flexible': cell_flex,
    'Cell_semitransparent': cell_semi,
    'Substrate_stack_sequence': subs,
    'ETL_stack_sequence': etl,
    'Perovskite_dimension_2D': per_2d,
    'Perovskite_dimension_2D3D_mixture': per_2d3d,
    'Perovskite_dimension_3D': per_3d,
    'Perovskite_dimension_3D_with_2D_capping_layer': per_3d2d_cap,
    'Perovskite_composition_perovskite_ABC3_structure': per_abc3,
    'Perovskite_composition_long_form': per,
    'Perovskite_composition_inorganic': per_ino,
    'Perovskite_band_gap_graded': per_band_grad,
    'HTL_stack_sequence': htl,
    'Backcontact_stack_sequence': back
})
dp_archi.shape

(27238400, 15)

In [13]:
# Importing the datapoints which showed improved efficiency from deposition optimization in Step-2
df_imp_pce_depo = pd.read_csv(pwd+"/predictions/improved_pce_depo.csv")
df_imp_pce_depo.shape

(5508977, 30)

In [14]:
df_imp_pce_depo.head()

,Cell_architecture,Cell_flexible,Cell_semitransparent,Substrate_stack_sequence,ETL_stack_sequence,ETL_thickness,ETL_deposition_procedure,Perovskite_dimension_2D,Perovskite_dimension_2D3D_mixture,Perovskite_dimension_3D,...,Perovskite_deposition_thermal_annealing_temperature,Perovskite_deposition_thermal_annealing_time,Perovskite_deposition_solvent_annealing,HTL_stack_sequence,HTL_thickness_list,HTL_deposition_procedure,Backcontact_stack_sequence,Backcontact_thickness_list,Backcontact_deposition_procedure,Predicted_PCE
0,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Spin-coating | Evaporation,False,False,True,...,100,20,False,NiO-c,344.0,Spin-coating,Cu,100.0,Evaporation,13.522444
1,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Spin-coating | Evaporation,False,False,True,...,100,20,False,NiO-c,344.0,Spin-coating,Cu,100.0,Evaporation | Evaporation,13.351723
2,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Spin-coating | Evaporation,False,False,True,...,100,20,False,NiO-c,344.0,Spin-coating,Cu,100.0,Sandwiching,12.840438
3,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Spin-coating | Evaporation,False,False,True,...,100,20,False,NiO-c,344.0,Spin-coating,Cu,100.0,Lamination,13.223305
4,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Spin-coating | Evaporation,False,False,True,...,100,20,False,NiO-c,344.0,Spin-coating | Spin-coating,Cu,100.0,Evaporation,13.415419


In [15]:
df_imp_pce_depo.sort_values(by="Predicted_PCE",ascending=False,inplace=True) # sorting

In [16]:
df_imp_pce_depo.head()

,Cell_architecture,Cell_flexible,Cell_semitransparent,Substrate_stack_sequence,ETL_stack_sequence,ETL_thickness,ETL_deposition_procedure,Perovskite_dimension_2D,Perovskite_dimension_2D3D_mixture,Perovskite_dimension_3D,...,Perovskite_deposition_thermal_annealing_temperature,Perovskite_deposition_thermal_annealing_time,Perovskite_deposition_solvent_annealing,HTL_stack_sequence,HTL_thickness_list,HTL_deposition_procedure,Backcontact_stack_sequence,Backcontact_thickness_list,Backcontact_deposition_procedure,Predicted_PCE
1177441,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Evaporation | Evaporation,False,False,True,...,60; 100,2.0; 10.0,False,NiO-c,344.0,Spin-coating | Spin-coating,Cu,100.0,Evaporation,15.247782
993403,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Evaporation | Evaporation,False,False,True,...,60; 100,2.0; 10.0,False,NiO-c,344.0,Spin-coating | Spin-coating,Cu,100.0,Evaporation,15.247782
1177201,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Evaporation | Evaporation,False,False,True,...,60; 100,1.0; 4.0,False,NiO-c,344.0,Spin-coating | Spin-coating,Cu,100.0,Evaporation,15.228935
993163,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Evaporation | Evaporation,False,False,True,...,60; 100,1.0; 4.0,False,NiO-c,344.0,Spin-coating | Spin-coating,Cu,100.0,Evaporation,15.228935
988137,pin,False,False,SLG | ITO,SnO2-np | CPTA,26.0,Evaporation | Evaporation,False,False,True,...,60.0; 100.0,2.0; 10.0,False,NiO-c,344.0,Spin-coating | Spin-coating,Cu,100.0,Evaporation,15.204399


In [17]:
# Taking the most frequent values of thickness and band_gap
for f in ['ETL_thickness', 'Perovskite_thickness','Perovskite_band_gap', 'HTL_thickness_list', 'Backcontact_thickness_list']:
    dp_archi[f] = df[f].mode()[0]

In [18]:
# All the features related to deposition is replaced with the values with data of highest PCE predicted from Step-2
for f in feats_deposition:
    dp_archi[f] = df_imp_pce_depo[f].iloc[0]

dp_archi = dp_archi[list(feats.columns)]
dp_archi.shape # final shape of the architecture design pool

(27238400, 29)

In [19]:
dp_archi.head()

,Cell_architecture,Cell_flexible,Cell_semitransparent,Substrate_stack_sequence,ETL_stack_sequence,ETL_thickness,ETL_deposition_procedure,Perovskite_dimension_2D,Perovskite_dimension_2D3D_mixture,Perovskite_dimension_3D,...,Perovskite_deposition_quenching_induced_crystallisation,Perovskite_deposition_thermal_annealing_temperature,Perovskite_deposition_thermal_annealing_time,Perovskite_deposition_solvent_annealing,HTL_stack_sequence,HTL_thickness_list,HTL_deposition_procedure,Backcontact_stack_sequence,Backcontact_thickness_list,Backcontact_deposition_procedure
0,pin,False,False,SLG | ITO,PCBM-60 | BCP,26.0,Evaporation | Evaporation,True,False,False,...,False,60; 100,2.0; 10.0,False,PEDOT:PSS,344.0,Spin-coating | Spin-coating,Ag,100.0,Evaporation
1,pin,False,False,SLG | ITO,PCBM-60 | BCP,26.0,Evaporation | Evaporation,True,False,False,...,False,60; 100,2.0; 10.0,False,PEDOT:PSS,344.0,Spin-coating | Spin-coating,Al,100.0,Evaporation
2,pin,False,False,SLG | ITO,PCBM-60 | BCP,26.0,Evaporation | Evaporation,True,False,False,...,False,60; 100,2.0; 10.0,False,PEDOT:PSS,344.0,Spin-coating | Spin-coating,Au,100.0,Evaporation
3,pin,False,False,SLG | ITO,PCBM-60 | BCP,26.0,Evaporation | Evaporation,True,False,False,...,False,60; 100,2.0; 10.0,False,PEDOT:PSS,344.0,Spin-coating | Spin-coating,Ag | Au,100.0,Evaporation
4,pin,False,False,SLG | ITO,PCBM-60 | BCP,26.0,Evaporation | Evaporation,True,False,False,...,False,60; 100,2.0; 10.0,False,PEDOT:PSS,344.0,Spin-coating | Spin-coating,Cu,100.0,Evaporation


In [20]:
pickle.dump(dp_archi, open(pwd+"/design_pool/design_pool_archi_v2.pkl","wb")) # saving design pool